# WalMart Sales Forecast
This notebook explores the sales data of 45 Wal-Mart locations. Exploritory data analysis was perfomed at the store and deparment level and the finding were used to forecast the sales in the future.

    Functionality Setting

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

    Dependencies

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import pandas as pd
import numpy as np
import datetime

# 1. Data Imports
The sales and store data was collected from [Walmart Recruiting's Kaggle Competition](https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting). This dataset only includes the influence of 4 holidays. Entering this project with the assumption that consumer spending is heavily influenced by holidays so a [second source](https://www.timeanddate.com/holidays/us/2010?hol=17) was utilized to collect all relevant consumer events and commonly observed U.S. holidays. The datasets were then consolidated in a [separate script](https://github.com/octaviaisom/Walmart-Sales-Forecast/blob/master/holidates.py) (due to its extended runtime) and saved in ``` sales_updated.csv ```.

In [19]:
csv = pd.read_csv("data/sales_updated.csv")
sales = pd.DataFrame(csv)

sales.tail()
sales.info()
sales.describe()

,Store,Dept,Date,Weekly_Sales,IsHoliday
421565,45,98,9/28/2012,508.37,False
421566,45,98,10/5/2012,628.10,False
421567,45,98,10/12/2012,1061.02,False
421568,45,98,10/19/2012,760.01,False
421569,45,98,10/26/2012,1076.80,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
Store           421570 non-null int64
Dept            421570 non-null int64
Date            421570 non-null object
Weekly_Sales    421570 non-null float64
IsHoliday       421570 non-null bool
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


,Store,Dept,Weekly_Sales
count,421570.000000,421570.000000,421570.000000
mean,22.200546,44.260317,15981.258123
std,12.785297,30.492054,22711.183519
min,1.000000,1.000000,-4988.940000
25%,11.000000,18.000000,2079.650000
50%,22.000000,37.000000,7612.030000
75%,33.000000,74.000000,20205.852500
max,45.000000,99.000000,693099.360000


In [7]:
sales.Date = pd.to_datetime(sales.Date)

In [8]:
csv = pd.read_csv("data/stores.csv")
stores = pd.DataFrame(csv)

stores.tail()
stores.info()
stores.describe()

,Store,Type,Size
40,41,A,196321
41,42,C,39690
42,43,C,41062
43,44,C,39910
44,45,B,118221


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
Store    45 non-null int64
Type     45 non-null object
Size     45 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


,Store,Size
count,45.000000,45.000000
mean,23.000000,130287.600000
std,13.133926,63825.271991
min,1.000000,34875.000000
25%,12.000000,70713.000000
50%,23.000000,126512.000000
75%,34.000000,202307.000000
max,45.000000,219622.000000


In [14]:
csv = pd.read_csv("data/holidays.csv")
holidays = pd.DataFrame(csv)

holidays.head()

,Holiday,Date
0,Black Friday,2010-11-26
1,Black Friday,2011-11-25
2,Black Friday,2012-11-23
3,Black Friday,2013-11-29
4,Christmas Day,2010-12-25


# 2. EDA

## Record Counts

In [20]:
sales.nunique()

Store               45
Dept                81
Date               143
Weekly_Sales    359464
IsHoliday            2
dtype: int64

In [23]:
45*81*143 == len(sales)

False

In [25]:
#Departments with missing weekly sales
sales.groupby(['Dept']).nunique().query("Date<143")

,Store,Dept,Date,Weekly_Sales,IsHoliday
Dept,,,,,
39,5,1,16,12,2
43,5,1,12,6,2
47,37,1,138,277,2
51,37,1,131,780,2
77,37,1,35,58,2
78,37,1,82,38,2
99,37,1,94,311,2


In [26]:
#Stores with missing weekly sales
sales.groupby(['Store']).nunique().query("Date<143")

,Store,Dept,Date,Weekly_Sales,IsHoliday
Store,,,,,


In [29]:
#Number of unique depts per store
sales.groupby(['Store']).nunique().sort_values(['Dept']).head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
Store,,,,,
43,1,61,143,6292,2
37,1,62,143,6864,2
42,1,62,143,6452,2
44,1,62,143,6548,2
33,1,63,143,5714,2


## Sales Analysis
High-level analysis of sales and holiday's potential influence on seasonality

In [ ]:
byDate = sales.groupby('Date').sum()
byDate = byDate.reset_index()
byDate.head()

In [ ]:
ax = byDate.plot(x='Date', y='Weekly_Sales', figsize=(15,5))

In [ ]:
first_yr = byDate.iloc[:52]

ax = first_yr.plot(x='Date', y='Weekly_Sales', figsize=(15,5))
ax.autoscale(axis='x',tight=True)

for index, row in first_yr.query("IsHoliday!=0").iterrows():
    holiday = row['Date']
    ax.axvline(x=holiday,color='k', alpha = 0.2)

# Sales by Store
in this section i aim to discover...how does size/type influence sales?

In [ ]:
#number of store
sales['Store'].nunique()

In [ ]:
#number of store types
stores['Type'].nunique()

In [ ]:
sales.groupby(['Store']).count()

compare all...bar total sales across the period

In [ ]:
byStore = sales.groupby(['Store']).sum()
byStore['Weekly_Sales'].plot(kind='bar',figsize=(15,5))

SOME STORE THAT OUTPERFORM OTHERS...POTENTIALLY INFLUENCED BY SIZE LOCATION ETC...

In [ ]:
fgrid = sns.FacetGrid(data=byStore, col='Type')
fgrid.map(plt.scatter,'Size', 'Weekly_Sales')

...bc previous does not provided the desired detail...dig deeper into size..cluster to further analyze the sales behavior across stores SIZES...

In [ ]:
#calculate % of total sales
byStore = pd.merge(byStore, stores, on="Store", how="left")
byStore.head()

In [ ]:
byStore.plot(x="Size", y="Weekly_Sales",kind='scatter',figsize=(15,5))

seaborn grid...types vs size and sales

In [ ]:
byStore["SalesPerSF"] = byStore['Weekly_Sales']/byStore['Size']
byStore.plot(x="Size", y="SalesPerSF",kind='scatter',figsize=(15,5))

size distribution...bins?

In [ ]:
sns.distplot(byStore['Size'], kde=False, bins=5)

Cluster Stores

In [ ]:
byStorex = byStore[['Weekly_Sales', 'Size','SalesPerSF']]

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=6)

In [ ]:
kmeans.fit(byStorex)

In [ ]:
#kmeans.cluster_centers_ OTHER KMEANS FEATURES?

In [ ]:
byStorex['Cluster'] = kmeans.labels_

In [ ]:
byStorex.head()

NOW THAT THE STORE ARE CLUSTERED...MORE MEANINGFUL GRAPHS

In [ ]:
fig, ax1 = plt.subplots()
ax1.scatter(byStore.Size, byStorex.Weekly_Sales, color='b',edgecolor = 'black', alpha = 0.7)
#ax1.set_ylabel('Total Population (100M)', color='b')
#ax1.set_xlabel('Year')

ax2 = ax1.twinx()
ax2.scatter(byStore.Size, byStorex.SalesPerSF, color='r',edgecolor = 'black', alpha = 0.4)
#ax2.set_ylabel('Median Incarcerated Population (%)', color='r')

#plt.title('Total Population vs. Incarceration Rate (Nationwide)')



plt.show()


In [ ]:
fgrid = sns.FacetGrid(data=byStorex, col='Cluster')
fgrid.map(plt.scatter,'Size', 'Weekly_Sales')

In [ ]:
#seaborn...hue=cluster...group by cluster?????

In [ ]:
#GRID: CLUSTER V HOLIDAY V SALES??

FINAL STORE SALES THOUGHTS...

# Sales by Dept
IN THIS SECTION...BLAH BLAH BLA...DEPT-LEVEL SALES...WHICH DEPARTMENTS ARE INFLUENCED BY SEASONALITY?

In [ ]:
#number of departments
sales['Dept'].nunique()

COMPARE ALL BAR

In [ ]:
byDept = sales.groupby(['Dept']).sum()
byDept['Weekly_Sales'].plot(kind='bar',figsize=(15,8))

negative sales (loss) observed...filter store with negative sales

In [ ]:
byDept.query('Weekly_Sales<=0')

In [ ]:
byDept.loc[47]['Weekly_Sales']/byDept['Weekly_Sales'].sum()

DICKEY-FULLER TEST

In [ ]:
LOOP THROUGHT DEPT AND ASSIGN STATIONARITY ...THIS WILL DETERMINE HOW THE DEPTS ARE LATER FORESCASTED

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
csv = pd.read_csv("data/sales.csv")
salesx = pd.DataFrame(csv)
#salesx.reset_index(inplace=True)
#salesx.set_index('Date', inplace=True)
salesx.index.freq = 'W'
salesx.head()

In [ ]:
salesx.query("Dept==96")

In [ ]:
salesx.groupby(['Store']).nunique().sort_values(['Weekly_Sales'])

In [ ]:
pd.set_option('display.max_rows', 600)
x = salesx.groupby(['Store','Dept']).nunique()
x

In [ ]:
x.query("Weekly_Sales<143")

In [ ]:
pd.set_option('display.max_rows', 500)
salesx.groupby(['Dept']).nunique().sort_values(['Weekly_Sales'])

In [ ]:
pd.set_option('display.max_rows', 500)
x = salesx.groupby(['Dept']).nunique()
x.query("Store<45|Date<143").sort_values(['Date','Store'])

In [ ]:
salesx.iloc[[1,2,3]]

In [ ]:
depts = salesx['Dept'].unique()
salesx['Stationarity'] = ""
for dept in depts:
    dept_sales = salesx.query(f"Dept=={dept}")
    indices = dept_sales['index'].unique()
    dftest = adfuller(dept_sales['Weekly_Sales'],autolag='AIC')
    p_value = dftest[1]

    if p_value <= 0.05:
        stationarity = 1 #stationary
    else:
        stationarity = 0 #non-stationary
        
    salesx['Stationarity'].iloc[indices] = stationarity

In [ ]:
statCount = salesx.groupby(['Stationarity',"Dept"]).sum()
statCount
#percent of sales??

In [ ]:
salesx.query("Dept==1").groupby("Date").sum().plot(y="Weekly_Sales")

In [ ]:
dept=96
dept_sales = salesx.query(f"Dept=={dept}")
dept_sales.head()

In [ ]:
dftest = adfuller(dept_sales['Weekly_Sales'],autolag='AIC')
p_value = dftest[1]

if p_value <= 0.05:
    stationarity = 1 #stationary
else:
    stationarity = 0 #non-stationary

stationarity

STATIONARY VS NON-STATIONARY...COUNT, %OF DEPT, %OF SALES DEPEND ON SEASONALITY/TRENDS??

In [ ]:
#GRID CHART????

# 3. Forecasting
describe methodology..stationarity/DFT results determine model type(ARIMA or SARIMAx)...Eget felis eget nunc lobortis mattis. Vulputate sapien nec sagittis aliquam malesuada bibendum. Adipiscing tristique risus nec feugiat in fermentum posuere urna. Sapien pellentesque habitant morbi tristique senectus et netus. Faucibus scelerisque eleifend donec pretium vulputate sapien nec. Eget egestas purus viverra

In [ ]:
#GROUP BY DEPTS OR SPLIT DATA?

original data only include...incorporae all US consumer spending event (super bowl, black friday) and other commonly observed holdays that would drive consumer spending...assuming all us stores!!!!
back to school...no exact date???